In [1]:
import re
import json
import hashlib
import string
from collections import Counter, defaultdict
from itertools import permutations, combinations, product, chain
from dataclasses import dataclass
from functools import lru_cache
from math import prod, sqrt, inf
from parse import parse
from typing import List, Tuple, NamedTuple, Set

import black
import jupyter_black

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)

In [2]:
# Helper functions inspired by https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2021.ipynb
def data(day, parser=str, sep="\n", example=False, print_lines=7) -> tuple:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    filename = f"2015/example-{day}.txt" if example else f"2015/input-{day}.txt"
    text = open(filename).read()
    entries = tuple(parser(section) for section in text.rstrip().split(sep))
    if print_lines:
        all_lines = text.splitlines()
        lines = all_lines[:print_lines]
        head = f"{filename} ➜ {len(text)} chars, {len(all_lines)} lines; first {len(lines)} lines:"
        dash = "-" * 100
        print(f"{dash}\n{head}\n{dash}")
        for line in lines:
            print(trunc(line))
        print(
            f"{dash}\nparse({day}) ➜ {len(entries)} entries:\n"
            f"{dash}\n{trunc(str(entries))}\n{dash}"
        )
    return entries


def trunc(s: str, left=70, right=25, dots=" ... ") -> str:
    """All of string s if it fits; else left and right ends of s with dots in the middle."""
    dots = " ... "
    return s if len(s) <= left + right + len(dots) else s[:left] + dots + s[-right:]


def numbers(text: str) -> List[int]:
    "Return positive and negative integers found."
    return [int(x) for x in re.findall(r"-?\d+", text)]

In [3]:
# Day 1: Not Quite Lisp

instructions = data(1, print_lines=False)[0]
floor = instructions.count("(") - instructions.count(")")
print(f"Part 1: {floor}")  # 74
floor = step = 0
while floor > -1:
    if instructions[step] == "(":
        floor += 1
    elif instructions[step] == ")":
        floor -= 1
    else:
        raise ValueError("Unknown instruction found", instructions[step])
    step += 1
print(f"Part 2: {step}")

Part 1: 74
Part 2: 1795


In [4]:
# Day 2: I Was Told There Would Be No Math
def dimensions(text: str) -> Tuple[int, int, int]:
    """Parse a string of form '2x3x4'"""
    return tuple(int(x) for x in text.split("x"))


def paper(box: Tuple[int, int, int]) -> int:
    """Area of a box + slack, i.e. area of smallest side"""
    length, width, height = box
    sides = 2 * length * width + 2 * width * height + 2 * height * length
    slack = min(length * width, width * height, height * length)
    return sides + slack


def bow(box: Tuple[int, int, int]) -> int:
    """Return volume of box"""
    return prod(box)


def wrap(box: Tuple[int, int, int]) -> int:
    """Shortest distance around a box's sides"""
    shortest_sides = sorted(box)[:2]
    return sum(side * 2 for side in shortest_sides)


def ribbon(box: Tuple[int, int, int]) -> int:
    return wrap(box) + bow(box)


boxes = data(2, parser=dimensions, print_lines=False)
print(f"Part 1: {sum(paper(box) for box in boxes)} square feet")  # 1606483
print(f"Part 2: {sum(ribbon(box) for box in boxes)} feet of ribbon")  # 3842356

Part 1: 1606483 square feet
Part 2: 3842356 feet of ribbon


In [5]:
# Day 3: Perfectly Spherical Houses in a Vacuum
def move_next(position: Tuple[int, int], step: str) -> Tuple[int, int]:
    row, column = position
    match step:
        case "^":
            return row - 1, column
        case ">":
            return row, column + 1
        case "<":
            return row, column - 1
        case "v":
            return row + 1, column


moves = open("2015/input-3.txt").read().strip()
# Part 1
house = (0, 0)
houses = Counter([house])
for move in moves:
    house = move_next(house, move)
    houses[house] += 1
print(
    f"Part 1: {sum(1 for present in houses.values())} houses receive at least one present"
)  # 2081

# Part 2
santa = robo = (0, 0)
houses = Counter([santa])
houses[robo] += 1
for order, move in enumerate(moves):
    if order % 2:
        robo = move_next(robo, move)
        houses[robo] += 1
    else:
        santa = move_next(santa, move)
        houses[santa] += 1
print(
    f"Part 2: {sum(1 for present in houses.values())} houses receive at least one present"
)  # 2341

Part 1: 2081 houses receive at least one present
Part 2: 2341 houses receive at least one present


In [6]:
# Day 4: The Ideal Stocking Stuffer
base = "iwrupvqb"
number = 1
candidate = base + str(number)
while not hashlib.md5(candidate.encode("utf-8")).hexdigest().startswith("00000"):
    number += 1
    candidate = base + str(number)
print(f"Part 1: {number}")  # 346386

# Part 2 takes 13 seconds to complete
while not hashlib.md5(candidate.encode("utf-8")).hexdigest().startswith("000000"):
    number += 1
    candidate = base + str(number)
    if number % 100000 == 0:
        print(".", end="")
print(f"\nPart 2: {number}")  # 9958218

Part 1: 346386
................................................................................................
Part 2: 9958218


In [7]:
# Day 5: Doesn't He Have Intern-Elves For This?
def number_of_vowels(text: str) -> int:
    VOWELS = "aeiou"
    vowels = 0
    for letter in text:
        if letter in VOWELS:
            vowels += 1
    return vowels


def two_consecutive_letter(text: str) -> bool:
    for cur, next in zip(text, text[1:]):
        if cur == next:
            return True
    return False


def forbidden_pairs(text: str) -> bool:
    FORBIDDEN = ("ab", "cd", "pq", "xy")
    for i, _ in enumerate(text):
        if text[i : i + 2] in FORBIDDEN:
            return True
    return False


def is_nice(text: str) -> bool:
    return (
        (number_of_vowels(text) > 2)
        and two_consecutive_letter(text)
        and not forbidden_pairs(text)
    )


def two_pairs(text: str) -> bool:
    for i in range(len(text) - 1):
        candidate = text[i : i + 2]
        for j in range(i + 2, len(text) - 1):
            if candidate == text[j : j + 2]:
                return True
    return False


def repeating_letter(text: str) -> bool:
    "Text contains a letter that repeats with exactly one other letter between them."
    for letter, repeat in zip(text, text[2:]):
        if letter == repeat:
            return True
    return False


texts = data(5, print_lines=False)
print(f"Part 1: {sum(is_nice(text) for text in texts)} nice strings")
print(
    f"Part 2: {sum((two_pairs(text) and (repeating_letter(text)) for text in texts))} nice strings"
)

Part 1: 236 nice strings
Part 2: 51 nice strings


In [8]:
# Day 6: Probably a Fire Hazard
def part1(instructions: list) -> int:
    def turn_on(r1, c1, r2, c2):
        for row in range(r1, r2 + 1):
            for column in range(c1, c2 + 1):
                lights[row][column] = True

    def turn_off(r1, c1, r2, c2):
        for row in range(r1, r2 + 1):
            for column in range(c1, c2 + 1):
                lights[row][column] = False

    def toggle(r1, c1, r2, c2):
        for row in range(r1, r2 + 1):
            for column in range(c1, c2 + 1):
                lights[row][column] = not lights[row][column]

    lights = [[False] * 1000 for _ in range(1000)]

    for (command, r1, c1, r2, c2) in instructions:
        match command:
            case "toggle":
                toggle(r1, c1, r2, c2)
            case "turn on":
                turn_on(r1, c1, r2, c2)
            case "turn off":
                turn_off(r1, c1, r2, c2)
            case _:
                raise ValueError("Unknown command:", command)

    return sum(row.count(True) for row in lights)


def part2(instructions) -> int:
    def turn_on(r1, c1, r2, c2):
        for row in range(r1, r2 + 1):
            for column in range(c1, c2 + 1):
                lights[row][column] += 1

    def turn_off(r1, c1, r2, c2):
        for row in range(r1, r2 + 1):
            for column in range(c1, c2 + 1):
                lights[row][column] = max(lights[row][column] - 1, 0)

    def toggle(r1, c1, r2, c2):
        for row in range(r1, r2 + 1):
            for column in range(c1, c2 + 1):
                lights[row][column] += 2

    lights = [[0] * 1000 for _ in range(1000)]

    for (command, r1, c1, r2, c2) in instructions:
        match command:
            case "toggle":
                toggle(r1, c1, r2, c2)
            case "turn on":
                turn_on(r1, c1, r2, c2)
            case "turn off":
                turn_off(r1, c1, r2, c2)
            case _:
                raise ValueError("Unknown command:", command)

    return sum(sum(row) for row in lights)


lines = open("2015/input-6.txt").read().strip().splitlines()

instructions = []
for line in lines:
    command, r1, c1, r2, c2 = parse("{} {:d},{:d} through {:d},{:d}", line)
    instructions.append((command, r1, c1, r2, c2))

print(f"Part 1: {part1(instructions)} turned on lights")  # 543903
print(f"Part 2: {part2(instructions)} turned on lights")  # 14687245

Part 1: 543903 turned on lights
Part 2: 14687245 turned on lights


In [9]:
# Day 7: Some Assembly Required
class Expression(NamedTuple):
    value: int = 0
    method: str = None
    a: str = ""
    b: str = ""


@lru_cache
def evaluate(exp: Expression):
    match exp.method:
        case "AND":
            if exp.a == "1":
                return 1 & evaluate(expressions[exp.b])
            else:
                return evaluate(expressions[exp.a]) & evaluate(expressions[exp.b])
        case "OR":
            return evaluate(expressions[exp.a]) | evaluate(expressions[exp.b])
        case "NOT":
            return evaluate(expressions[exp.a]) ^ 0xFFFF
        case "LSHIFT":
            return evaluate(expressions[exp.a]) << int(exp.b)
        case "RSHIFT":
            return evaluate(expressions[exp.a]) >> int(exp.b)
        case "EQ":
            return evaluate(expressions[exp.a])
        case None:
            return exp.value
        case _:
            raise ValueError


def parse_expression(exp: str) -> Expression:
    if number := parse("{:d}", exp):
        return Expression(value=number[0])
    elif not_exp := parse("NOT {}", exp):
        return Expression(method="NOT", a=not_exp[0])
    elif other_exp := parse("{} {} {}", exp):
        (a, method, b) = other_exp
        return Expression(method=method, a=a, b=b)
    else:
        # Only one case in my input: lx -> a
        return Expression(method="EQ", a=exp)


expressions = defaultdict(Expression)
lines = data(7, example=False, print_lines=False)
for line in lines:
    expression, name = line.split(" -> ")
    expressions[name] = parse_expression(expression)

a = evaluate(expressions["a"])
print(f"Part 1: {a}")

expressions["b"] = Expression(value=a)
evaluate.cache_clear()
a = evaluate(expressions["a"])
print(f"Part 2: {a}")

Part 1: 16076
Part 2: 2797


In [10]:
# Day 8: Matchsticks
def encoded_len(line: str) -> int:
    """Count all '"' and '\' in `line` twice"""
    # Add 2 to account for first and last '"'
    return len(line) + line.count("\\") + line.count('"') + 2


lines = open("2015/input-8.txt").read().splitlines()
part1 = sum(len(line) - len(eval(line)) for line in lines)
print(f"Part 1: {part1}")
part2 = sum(encoded_len(line) - len(line) for line in lines)
print(f"Part 2: {part2}")

Part 1: 1342
Part 2: 2074


In [11]:
# Day 9: All in a Single Night
distances = defaultdict(int)
cities = set()
for line in open("2015/input-9.txt").read().strip().splitlines():
    *pair, dist = parse("{} to {} = {:d}", line)
    distances[tuple(sorted(pair))] = dist
    cities.update(pair)

paths = defaultdict(int)
# All cities are connected to each other, permutations gives all possible paths visiting
# each city once
for path in permutations(cities):
    for pair in zip(path, path[1:]):
        pair = sorted(list(pair))
        pair = tuple(pair)
        paths[path] += distances[pair]

print(f"Part 1: {min(dist for dist in paths.values())}")  # 251
print(f"Part 2: {max(dist for dist in paths.values())}")  # 898

Part 1: 251
Part 2: 898


In [12]:
# Day 10: Elves Look, Elves Say
def look_say(seq: str) -> str:
    "Implements https://en.wikipedia.org/wiki/Look-and-say_sequence"
    count = 1
    new_seq = ""
    for prev, cur in zip(seq, seq[1:]):
        if prev == cur:
            count += 1
        else:
            new_seq += str(count) + prev
            count = 1
    return new_seq + str(count) + cur


sequence = "1321131112"

for i in range(40):
    sequence = look_say(sequence)
print(f"Part 1: {len(sequence)}")  # 492982

for i in range(10):
    sequence = look_say(sequence)
print(f"Part 2: {len(sequence)}")  # 6989950

Part 1: 492982
Part 2: 6989950


In [13]:
# Day 11: Corporate Policy
def increment_line(line: str) -> str:
    """
    Increment line by increasing the last character one step.
    If the last character becomes 'a' then the second to last is increased, etc.
    """

    def increment_character(char: str) -> str:
        FORBIDDEN = "iol"
        # Convert to base 36. a=10, b=11, ... z=35
        number = int(char, 36) - 10
        number = (number + 1) % 26
        new_char = string.ascii_lowercase[number]
        if new_char in FORBIDDEN:
            number += 1
            return string.ascii_lowercase[number]
        return new_char

    new_line = []
    for i in range(len(line) - 1, -1, -1):
        new_char = increment_character(line[i])
        new_line.append(new_char)
        if new_char != "a":
            break
    for j in range(i - 1, -1, -1):
        new_line.append(line[j])
    return "".join(reversed(new_line))


def three_character_straight(line: str) -> bool:
    numbers = [int(char, 36) - 10 for char in line]
    for i in range(len(numbers) - 2):
        if numbers[i] == (numbers[i + 1] - 1) and (numbers[i] == (numbers[i + 2] - 2)):
            return True
    return False


def two_pairs(line: str) -> bool:
    "`line` contains two non-overlapping pairs."
    one_pair = False
    zipped = zip(line, line[1:])
    for a, b in zipped:
        if a == b:
            if not one_pair:
                one_pair = True
                next(zipped, None)
            else:
                return True
    return False


def next_valid_password(password: str) -> str:
    def is_correct(password: str) -> bool:
        return two_pairs(password) and three_character_straight(password)

    password = increment_line(password)
    while not is_correct(password):
        password = increment_line(password)
    return password


password = "hxbxwxba"
password = next_valid_password(password)
print(f"Part 1: {password}")  # hxbxxyzz
print(f"Part 2: {next_valid_password(password)}")  # hxcaabcc

Part 1: hxbxxyzz
Part 2: hxcaabcc


In [14]:
# Day 12: JSAbacusFramework.io
def discard_red(dct):
    if "red" in dct.values():
        return 0
    return dct


db = open("2015/input-12.txt").read()
print(f"Part 1: {sum(num for num in numbers(db))}")  # 156366

db_dict = json.loads(db, object_hook=discard_red)
print(f"Part 2: {sum(num for num in numbers(json.dumps(db_dict)))}")  # 96852

Part 1: 156366
Part 2: 96852


In [15]:
# Day 13: Knights of the Dinner Table
def seating_points(seating: tuple) -> int:
    points = 0
    for i, guest in enumerate(seating):
        left = seating[i - 1]
        right = seating[(i + 1) % len(seating)]
        points += happiness[(guest, left)] + happiness[(guest, right)]
    return points


guests = set()
happiness = defaultdict(int)
for line in open("2015/input-13.txt").read().splitlines():
    line = line.replace("lose ", "-").replace("gain ", "")
    name, points, neighbor = parse(
        "{} would {:d} happiness units by sitting next to {}.", line
    )
    happiness[(name, neighbor)] = points
    guests.add(name)
guests = list(guests)

# Can be optimized by fixing the first position and only permutate over the remainder.
max_happiness = max(seating_points(seating) for seating in permutations(guests))
print(f"Part 1: {max_happiness}")  # 664

for guest in guests:
    happiness[("me", guest)] = 0
guests.append("me")
max_happiness = max(seating_points(seating) for seating in permutations(guests))
print(f"Part 2: {max_happiness}")  # 640

Part 1: 664
Part 2: 640


In [16]:
# Day 14: Reindeer Olympics
@dataclass
class Reindeer:
    speed: int
    fly: int
    rest: int
    seconds: int = 0
    position: int = 0
    score: int = 0

    def tick(self):
        if self.seconds in range(self.fly):
            self.position += self.speed
        self.seconds = (self.seconds + 1) % (self.fly + self.rest)


def leader(reindeer: List[Reindeer]) -> Reindeer:
    lead_pos = max(rein.position for rein in reindeer)
    for rein in reindeer:
        if rein.position == lead_pos:
            return rein


reindeer = []
for line in open("2015/input-14.txt").read().splitlines():
    reindeer.append(Reindeer(*numbers(line)))

for _ in range(2503):
    for rein in reindeer:
        rein.tick()
    leader(reindeer).score += 1
print(f"Part 1: {max(rein.position for rein in reindeer)}")  # 2655
print(f"Part 2: {max(rein.score for rein in reindeer)}")  # 1059

Part 1: 2655
Part 2: 1059


In [17]:
# Day 15: Science for Hungry People
def get_scores(
    a: List[int], b: List[int], c: List[int], d: List[int]
) -> Tuple[int, int]:
    teaspoons = 100
    scores = []
    slim_scores = []
    for x in range(teaspoons):
        for y in range(teaspoons - x):
            for z in range(teaspoons - x - y):
                score = [
                    a[i] * x + b[i] * y + c[i] * z + d[i] * (teaspoons - x - y - z)
                    for i in range(len(a))
                ]

                if not any(map(lambda x: x < 0, score)) and (
                    result := prod(score[:-1])
                ):
                    scores.append(result)
                    if score[-1] == 500:
                        slim_scores.append(result)

    return max(scores), max(slim_scores)


ingredients = []
for line in open(("2015/input-15.txt")):
    ingredients.append(numbers(line))
part1, part2 = get_scores(*ingredients)
print(f"Part 1: {part1}")  # 18965440
print(f"Part 2: {part2}")  # 15862900

Part 1: 18965440
Part 2: 15862900


In [ ]:
# Day 15 - generic nested for loop from reddit
def mixtures(depth, total):
    "A nested loop that is `depth` deep and sums up to `total`"
    start = total if depth == 1 else 0

    for i in range(start, total + 1):
        remains = total - i
        if depth - 1:
            for y in mixtures(depth - 1, remains):
                yield [i] + y
        else:
            yield [i]

In [19]:
# Day 16: Aunt Sue
def match_sue(sue: defaultdict) -> bool:
    return all(sue[k] == target_sue[k] for k in sue.keys())


def match_sue2(sue: defaultdict) -> bool:
    def match(key: str):
        if key in ("cats", "trees"):
            return sue[key] > target_sue[key]
        elif key in ("pomeranians", "goldfish"):
            return sue[key] < target_sue[key]
        else:
            return sue[key] == target_sue[key]

    return all(match(key) for key in sue.keys())


target_sue = {
    "children": 3,
    "cats": 7,
    "samoyeds": 2,
    "pomeranians": 3,
    "akitas": 0,
    "vizslas": 0,
    "goldfish": 5,
    "trees": 3,
    "cars": 2,
    "perfumes": 1,
}


aunt_sues = []
for line in open("2015/input-16.txt").read().splitlines():
    _, ak, av, bk, bv, ck, cv = parse("Sue {:d}: {}: {:d}, {}: {:d}, {}: {:d}", line)
    aunt_sues.append({ak: av, bk: bv, ck: cv})

for i, sue in enumerate(aunt_sues):
    if match_sue(sue):
        print(f"Part 1: {i+1}")  # 373
    if match_sue2(sue):
        print(f"Part 2: {i+1}")  # 260

Part 2: 260
Part 1: 373


In [20]:
# Day 17: No Such Thing as Too Much
lines = open("2015/input-17.txt").read().splitlines()
containers = [int(x) for x in lines]

solutions = [
    combo
    for no_of_cont in range(1, len(containers) + 1)
    for combo in combinations(containers, no_of_cont)
    if sum(combo) == 150
]

print(f"Part 1: {len(solutions)}")  # 4372

min_containers = len(min(solutions, key=lambda x: len(x)))
num_min_combos = sum(1 for sol in solutions if len(sol) == min_containers)
print(f"Part 2: {num_min_combos}")  # 4

Part 1: 4372
Part 2: 4


In [ ]:
# Day 17: Alternative implenentations

# Dynamic programming
def ways_to_fill(containers, target):
    # Index is volume, value ways to fill to that volume.
    # When volume is 0, the possible ways to fill is 1
    ways_to_fill = [1] + [0] * target
    for container in containers:
        for volume in range(target, container - 1, -1):
            ways_to_fill[volume] += ways_to_fill[volume - container]
    return ways_to_fill[target]


ways_to_fill([20, 15, 10, 5, 5], 25)

# Day 17: Recursion.
# Could use @lru_cache to speed it up, but not needed for this problem space
def combos(volume, containers):
    if volume == 0:
        return 1  # Contains all the water
    if volume < 0 or len(containers) == 0:
        return 0  # No solution, don't explore further
    candidate = containers[0]
    tail = containers[1:]
    # Return combos including and excluding candidate
    return combos(volume - candidate, tail) + combos(volume, tail)


# combos(25, [20, 15, 10, 5, 5])
combos(150, containers)

# Day 17: Recursion with yield
def combos(target, containers):
    for i, candidate in enumerate(containers):
        # This check reduces number of method calls from 1,000k to 166k
        if target - candidate < 0:
            continue
        remainder = target - candidate
        if remainder:
            for tail in combos(remainder, containers[i + 1 :]):
                yield [candidate] + tail
        else:
            yield [candidate]


solutions = list(combos(150, containers))
print(len(solutions))
min_containers = len(min(solutions, key=lambda x: len(x)))
sum(1 for sol in solutions if len(sol) == min_containers)

In [24]:
# Aside: Fibonacci three ways: recursion, memoized and dynamic programming
def fib_rec(n):
    # fib(37)  # 24157817 Takes ~6 s
    if n == 1 or n == 2:
        return 1
    return fib_rec(n - 1) + fib_rec(n - 2)


@lru_cache
def fib_cache(n):
    # fib_cache(37)  # 24157817, takes 0.4 s
    if n == 1 or n == 2:
        return 1
    return fib_cache(n - 1) + fib_cache(n - 2)


def fib_dp(n):
    # fib_dp(37)  # 24157817, 0.4 s
    fib = defaultdict(int)
    fib[1] = fib[2] = 1
    for x in range(3, n + 1):
        fib[x] = fib[x - 2] + fib[x - 1]
    return fib[n]

In [25]:
# Day 18: Like a GIF For Your Yard
def parse_grid():
    lines = open("2015/input-18.txt").read().strip().splitlines()
    return {
        (row, column)
        for row, line in enumerate(lines)
        for column, light in enumerate(line)
        if light == "#"
    }


def max_coord():
    lines = open("2015/input-18.txt").read().strip().splitlines()
    return len(lines[0]), len(lines)


def lit_neighbors(point, grid):
    x, y = point
    candidates = [
        (x - 1, y - 1),
        (x - 1, y),
        (x - 1, y + 1),
        (x, y - 1),
        (x, y + 1),
        (x + 1, y - 1),
        (x + 1, y),
        (x + 1, y + 1),
    ]
    return [point for point in candidates if point in grid]


def update_grid(grid):
    # Lights that were on and have 2 neighbors should be left on
    still_on = {point for point in grid if len(lit_neighbors(point, grid)) == 2}
    # All lights with 3 neighbors should be turned on no matter previous status
    turned_on = {point for point in all_points if len(lit_neighbors(point, grid)) == 3}
    return still_on | turned_on


grid = parse_grid()
max_x, max_y = max_coord()
all_points = {point for point in product(range(max_x), range(max_y))}
for _ in range(100):
    grid = update_grid(grid)
print(f"Part 1: {len(grid)}")  # 814

always_on = {(0, 0), (max_x - 1, 0), (0, max_y - 1), (max_x - 1, max_y - 1)}
grid = parse_grid() | always_on
for _ in range(100):
    grid = update_grid(grid) | always_on
print(f"Part 2: {len(grid)}")  # 924

Part 1: 814
Part 2: 924


In [26]:
# Day 19: Medicine for Rudolph
*rules, target = open("2015/input-19.txt").read().strip().splitlines()
rules.pop()
replacements = [tuple(rule.split(" => ")) for rule in rules]

molecule = re.findall(r"[A-Z][a-z]?", target)
molecules = set()
for replacement in replacements:
    for i, atom in enumerate(molecule):
        if atom == replacement[0]:
            new_molecule = molecule.copy()
            new_molecule[i] = replacement[1]
            molecules.add("".join(new_molecule))
print(f"Part 1: {len(molecules)}")

# We want to replace longest substring first, reverse sort by length
replacements = sorted(replacements, key=lambda x: -len(x[1]))
steps = 0
while target != "e":
    for atom, replacement in replacements:
        if target.find(replacement) >= 0:
            target = target.replace(replacement, atom, 1)
            steps += 1
            break
print(f"Part 2: {steps}")

Part 1: 518
Part 2: 200


In [27]:
# Day 20: Infinite Elves and Infinite Houses
# https://oeis.org/A000203

# 14 seconds for both parts, same implementation as before, but using numpy array with
# start:stop:step indexing for inner loop.
# https://blog.jverkamp.com/2015/12/20/advent-of-code-day-20/

import numpy as np

goal = 29_000_000
max_house = goal // 10

houses = np.zeros(max_house)
for elf in range(1, max_house):
    houses[elf::elf] += 10 * elf

house = 0
while houses[house] <= goal:
    house += 1
print(f"Part 1: {house}")  # 665280

# Part 2
houses = np.zeros(max_house)
for elf in range(1, max_house):
    houses[elf : elf * 51 : elf] += 11 * elf

house = 0
while houses[house] <= goal:
    house += 1
print(f"Part 2: {house}")  # 705600

Part 1: 665280
Part 2: 705600


In [28]:
# Day 21: RPG Simulator 20XX
@dataclass
class Warrior:
    hit_points: int
    damage: int
    armor: int


class Item(NamedTuple):
    cost: int = 0
    damage: int = 0
    armor: int = 0


def fight(attacker: Warrior, defender: Warrior):
    defender.hit_points -= max(attacker.damage - defender.armor, 1)


def player_wins(player: Warrior, boss: Warrior) -> bool:
    "Returns true if player wins the fight, false if boss wins"
    while player.hit_points > 0:
        fight(player, boss)
        if boss.hit_points <= 0:
            return True
        fight(boss, player)
    return False


def get_loadouts() -> List[Tuple[Item]]:
    """A list sorted by cost with all combinations of exactly one weapon, 0-1 armor and
    0-2 rings."""
    weapons = [
        Item(8, 4, 0),
        Item(10, 5, 0),
        Item(25, 6, 0),
        Item(40, 7, 0),
        Item(74, 8, 0),
    ]

    armors = [
        Item(13, 0, 1),
        Item(31, 0, 2),
        Item(53, 0, 3),
        Item(75, 0, 4),
        Item(102, 0, 5),
    ]

    rings = [
        Item(25, 1),
        Item(50, 2),
        Item(100, 3),
        Item(20, 0, 1),
        Item(40, 0, 2),
        Item(80, 0, 3),
    ]

    return sorted(
        [
            (
                weapon.cost + sum(a.cost for a in armor) + sum(r.cost for r in ring),
                (weapon,),
                armor,
                ring,
            )
            for weapon in weapons
            for i in (0, 1)
            for armor in combinations(armors, i)
            for j in (0, 1, 2)
            for ring in combinations(rings, j)
        ]
    )


def equip_loadout(player, loadout):
    for items in loadout:
        if isinstance(items, int):
            continue
        for item in items:
            player.damage += item.damage
            player.armor += item.armor


loadouts = get_loadouts()

for loadout in loadouts:
    boss = Warrior(100, 8, 2)
    player = Warrior(100, 0, 0)
    equip_loadout(player, loadout)
    if player_wins(player, boss):
        break
print(f"Part 1: {loadout[0]}")  # 91

for loadout in loadouts[::-1]:
    boss = Warrior(100, 8, 2)
    player = Warrior(100, 0, 0)
    equip_loadout(player, loadout)
    if not player_wins(player, boss):
        break
print(f"Part 2: {loadout[0]}")  # 158

Part 1: 91
Part 2: 158


In [29]:
# Day 22: Wizard Simulator 20XX
class GameState(NamedTuple):
    player: int
    mana: int
    boss: int
    damage: int
    shield: int = 0
    poison: int = 0
    recharge: int = 0


PRICES = {0: 53, 1: 73, 2: 113, 3: 173, 4: 229}


def player_round(state: GameState, spell: int, hard: bool) -> GameState:
    """
    0: Magic Missile costs 53 mana. It instantly does 4 damage.
    1: Drain costs 73 mana. It instantly does 2 damage and heals you for 2 hit points.
    2: Shield costs 113 mana. It starts an effect that lasts for 6 turns. While it is active, your armor is increased by 7.
    3: Poison costs 173 mana. It starts an effect that lasts for 6 turns. At the start of each turn while it is active, it deals the boss 3 damage.
    4: Recharge costs 229 mana. It starts an effect that lasts for 5 turns. At the start of each turn while it is active, it gives you 101 new mana.
    """
    player, mana, boss, damage, shield, poison, recharge = state
    if hard:
        player -= 1
        if player <= 0:
            # Player dies
            return GameState(player, mana, boss, damage, shield, poison, recharge)

    # Active effects
    if shield:
        # Shield only has effect during boss round, but ticks down here
        shield -= 1
    if poison:
        boss -= 3
        poison -= 1
    if recharge:
        mana += 101
        recharge -= 1

    # Spells
    # fmt: off
    match spell:
        case 0: boss -= 4
        case 1: player += 2; boss -= 2
        case 2: shield = 6
        case 3: poison = 6
        case 4: recharge = 5
    # fmt: on
    mana -= PRICES[spell]
    return GameState(player, mana, boss, damage, shield, poison, recharge)


def boss_round(state: GameState) -> GameState:
    "Return False if boss dies, otherwise new game state"
    player, mana, boss, damage, shield, poison, recharge = state
    # Active effects
    if shield:
        player += 7
        shield -= 1
    if poison:
        boss -= 3
        poison -= 1
        if boss <= 0:
            # Boss is dead, can't inflict any damage to player
            return GameState(player, mana, boss, damage, shield, poison, recharge)
    if recharge:
        mana += 101
        recharge -= 1
    player -= damage
    return GameState(player, mana, boss, damage, shield, poison, recharge)


def possible_spells(game: GameState) -> Set[GameState]:
    can_afford = set(spell for spell in PRICES if game.mana >= PRICES[spell])
    active_effects = set()
    if game.shield > 1:
        active_effects.add(2)
    if game.poison > 1:
        active_effects.add(3)
    if game.recharge > 1:
        active_effects.add(4)
    return can_afford - active_effects


@lru_cache(maxsize=None)
def play_rounds(game: GameState, hard: bool) -> int:

    best = inf

    for spell in possible_spells(game):
        state = player_round(game, spell, hard)
        if state.player > 0 and state.boss <= 0:
            return PRICES[spell]

        state = boss_round(state)
        if state.player > 0 and state.boss <= 0:
            return PRICES[spell]
        elif state.player <= 0:
            return inf

        new_cost = PRICES[spell] + play_rounds(state, hard)
        if new_cost < best:
            best = new_cost

    return best


state = GameState(player=50, mana=500, boss=55, damage=8)
print(f"Part 1: {play_rounds(state, hard=False)}")  # 953
print(f"Part 2: {play_rounds(state, hard=True)}")  # 1289

Part 1: 953
Part 2: 1289


In [30]:
# Day 23: Opening the Turing Lock
def execute_instruction(instructions, pc, a, b):
    instruction = instructions[pc]
    # All operations in my input are on register a except for inc
    match instruction:
        case "hlf", _:
            a = a // 2
            pc += 1
        case "tpl", _:
            a = a * 3
            pc += 1
        case "inc", reg:
            if reg == "a":
                a += 1
            else:
                b += 1
            pc += 1
        case "jmp", offset:
            pc += int(offset)
        case "jie", _, offset:
            if a % 2 == 0:
                pc += int(offset)
            else:
                pc += 1
        case "jio", _, offset:
            if a == 1:
                pc += int(offset)
            else:
                pc += 1
        case _:
            raise ValueError("No match found", instruction)
    return pc, a, b


lines = open("2015/input-23.txt").readlines()
instructions = tuple(tuple(line.split()) for line in lines)

pc, a, b = 0, 0, 0
while pc < len(instructions):
    pc, a, b = execute_instruction(instructions, pc, a, b)
print(f"Part 1: {b}")  # 307

pc, a, b = 0, 1, 0
while pc < len(instructions):
    pc, a, b = execute_instruction(instructions, pc, a, b)
print(f"Part 2: {b}")  # 160

Part 1: 307
Part 2: 160


In [33]:
# Day 24: It Hangs in the Balance
lines = open("2015/input-24.txt").readlines()
packages = [int(x) for x in lines]
target = sum(packages) / 3

# Find combinations with fewest number of packages, return smallest product
for n in range(1, len(packages)):
    good = [x for x in combinations(packages, n) if sum(x) == target]
    if len(good) > 0:
        break
print(f"Part 1: {min(prod(x) for x in good)}") # 10723906903

target = sum(packages) / 4  # 384
for n in range(1, len(packages)):
    good = [x for x in combinations(packages, n) if sum(x) == target]
    if len(good) > 0:
        break
print(f"Part 2: {min(prod(x) for x in good)}") # 74850409

Part 1: 10723906903
Part 2: 74850409


In [34]:
# Day 25: Let It Snow
def triangle_number(x: int):
    return x * (x + 1) // 2


def next_code(prev: int):
    return prev * factor % denominator


def sequence_number(column: int, row: int):
    return triangle_number(column) + column * (row - 1) + triangle_number(row - 2)


first = 20151125
factor = 252533
denominator = 33554393

# To continue, please consult the code grid in the manual.
# Enter the code at row 2947, column 3029.
row = 2947
column = 3029

seq = sequence_number(column, row)
code = first
# Takes 5 seconds to execute
for i in range(1, seq):
    code = next_code(code)
print(f"Part 1: {code}")

# Instant solution using pow which implements modular exponentiation
# print(first * pow(factor, seq - 1, denominator) % denominator)

Part 1: 19980801
